In [7]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
buy_times = 0
absolute_profit = 0
absolute_loss = 0

In [8]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [9]:
symbols = """
select distinct(name) from stockdataweekly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [10]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [11]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    training = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdataweekly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)



<h1> Updating resources

In [12]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdataweekly where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done


KeyboardInterrupt: 

In [ ]:
sql_dates = """
select distinct(date) from stockdataweekly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [ ]:

def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global absolute_loss
    global absolute_profit
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdataweekly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    
                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        
                        current_symbol.bought = current_price
                        current_symbol.stop_order = current_symbol.support * 0.97
                        quantity = (0.02 * capital) / current_price
                        remaining_cash = remaining_cash - commission - (quantity * current_price)                        
                        position[symbol]=quantity
                        current_symbol.times_bought += 1
                        print("Buying ", quantity, "stocks of ", symbol, "At price", current_symbol.bought, "on date", date)
                        training_inst = training_set_extractor(data_till_date, date)                            
                        current_symbol.training.append(training_inst)
                        
                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                current_symbol.training[-1][-1] = 1
                                win += 1
                                current_symbol.flag_for_wins = True
                        
                        
                    if current_symbol.stop_order > current_price:
                        
                        
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            current_symbol.training[-1][-1]= 1
                            if current_symbol.flag_for_wins == False:
                                win += 1
                            current_symbol.flag_for_wins = False
                            absolute_profit += profit_today
                        else:
                            loss += 1
                            current_symbol.training[-1][-1] = 0
                            current_symbol.flag_for_wins = False
                            absolute_loss += profit_today
                        profit += profit_today
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [ ]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    past_ten_weeks = data.tail(10)
    difference = abs(len(past_ten_weeks['2. high'].tolist()) - 10)
    zeros_list = [0] * (difference)
    training_instance = [zeros_list[:], zeros_list[:], 'na']
        
    training_instance[0].extend(past_ten_weeks['2. high'].tolist()[:])
    training_instance[1].extend(past_ten_weeks['5. volume'].tolist()[:])
    return training_instance[:]
   

In [ ]:
automatic_trader(symbol_list)

<h1> Extracting training sets from the trading strategy's cache

In [ ]:
training = []
for i in symbol_list:
    if(stock_equities[i].training):
        if stock_equities[i].training[-1][-1] == 'na':
            del stock_equities[i].training[-1]
        training.extend(stock_equities[i].training)
        
X = []
Y = []


random.shuffle(training)


for i in training:
    X.append([i[0], i[1]])
    Y.append([i[2]])

<h1> Saving training sets 

In [ ]:
import pickle
pickle_out = open("Y10Weekly.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()


pickle_out = open("X10Weekly.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [25]:
len(Y)

2386

In [26]:
win

965

In [15]:
tr = [stock_equities[i].training for i in symbol_list if len(stock_equities[i].training) > 0]

In [27]:
len(tr)

NameError: name 'tr' is not defined

In [21]:
987/2435

0.4053388090349076

In [1]:
987/2435

0.4053388090349076